## facial point 정보를 수집하여 csv로 저장

* image-url로 부터 facial point 수집하는 api

In [1]:
from __future__ import print_function

In [2]:
#import urllib.parse
#import urllib.request
import json
import requests

## face detect api call 
def face_detect(imgurl):
    url = 'http://apius.faceplusplus.com/v2/detection/detect'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'attribute' : 'none',
              'url' : imgurl,
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
  
    response = requests.post(url,params=values,headers=headers)
    response_data = response.text
    return json.loads(response_data.decode('utf-8'))

In [3]:
face_detect('http://historie.kr/img/face/goodvc_01.jpg')

{u'face': [{u'attribute': {},
   u'face_id': u'458ea1e109360c3706bd1571bea15ad6',
   u'position': {u'center': {u'x': 46.041667, u'y': 27.5},
    u'eye_left': {u'x': 42.326667, u'y': 23.876583},
    u'eye_right': {u'x': 49.644583, u'y': 23.455722},
    u'height': 18.333333,
    u'mouth_left': {u'x': 43.7025, u'y': 32.486944},
    u'mouth_right': {u'x': 49.655625, u'y': 31.842778},
    u'nose': {u'x': 45.549167, u'y': 28.540833},
    u'width': 13.75},
   u'tag': u''}],
 u'img_height': 360,
 u'img_id': u'56c29ad9ee2b8e23b16f4ce841b612c4',
 u'img_width': 480,
 u'session_id': u'8eb8a46ba7dc41e9b2b2513574ee2760',
 u'url': u'http://historie.kr/img/face/goodvc_01.jpg'}

In [4]:
## face landmark api call by face_id
def face_landmark(faceid):
    url = 'http://apius.faceplusplus.com/v2/detection/landmark'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'face_id' : faceid,
              'type' : '83p',
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
    
    response = requests.post(url,params=values,headers=headers)
    response_data = response.text
    return json.loads(response_data.decode('utf-8'))

def  extract_face_landmark( imgurl, face_index=0 ) :
    fd = face_detect(imgurl)
    face_id = fd['face'][face_index]['face_id']
    return face_landmark(face_id)
    

In [5]:
extract_face_landmark('http://historie.kr/img/face/goodvc_01.jpg')

{u'result': [{u'face_id': u'b71fd3975abed24243b070bf266e0965',
   u'landmark': {u'contour_chin': {u'x': 47.067083, u'y': 39.401944},
    u'contour_left1': {u'x': 39.458125, u'y': 24.21875},
    u'contour_left2': {u'x': 39.194583, u'y': 26.481556},
    u'contour_left3': {u'x': 39.434583, u'y': 28.661111},
    u'contour_left4': {u'x': 39.767917, u'y': 30.751111},
    u'contour_left5': {u'x': 40.352083, u'y': 32.863889},
    u'contour_left6': {u'x': 41.308958, u'y': 34.540833},
    u'contour_left7': {u'x': 42.38625, u'y': 36.264167},
    u'contour_left8': {u'x': 43.623333, u'y': 37.763611},
    u'contour_left9': {u'x': 45.142083, u'y': 39.002222},
    u'contour_right1': {u'x': 54.243958, u'y': 22.742722},
    u'contour_right2': {u'x': 54.4525, u'y': 24.960806},
    u'contour_right3': {u'x': 54.390833, u'y': 27.675639},
    u'contour_right4': {u'x': 54.262917, u'y': 30.14},
    u'contour_right5': {u'x': 53.841042, u'y': 32.415833},
    u'contour_right6': {u'x': 53.051875, u'y': 34.535278},

In [6]:
import json
import numpy as np
from scipy.spatial import distance

def pos_distance(l,r):
#    print(l,r)
    return round(distance.euclidean( [l['x'], l['y']], [r['x'], r['y']] ),5)

def pos_center(l,r):
    return {'x':(l['x']+r['x'])/2, 'y':(l['y']+r['y'])/2}

def extract_face_feature(facial_info):
    cal_items = {'face_width'    :['contour_left9', 'contour_right9']
                 ,'face_height'  :['eyebrow_center','contour_chin']
                 ,'mouth_width'  :['mouth_left_corner','mouth_right_corner']  
                 ,'nose_width'   :['nose_left','nose_right']  
                 ,'eye_width'    :['left_eye_left_corner','right_eye_right_corner']  
                 ,'eye_to_eye'   :['left_eye_center','right_eye_center']  
                 ,'eyebrow_width':['left_eyebrow_upper_middle','left_eyebrow_lower_middle']  
                 ,'lip_to_chin'  :['mouth_lower_lip_bottom','contour_chin']  
                 ,'lip_to_nose'  :['nose_tip','mouth_upper_lip_top']}
    
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    ## add center position
    facial_pos['eyebrow_center'] = pos_center( facial_pos['left_eyebrow_upper_middle'], facial_pos['right_eyebrow_upper_middle']) 
    
    ## calculate key point
    for (k,v) in cal_items.items(): ## iteritems() for python2.7
        keypoint[k] = pos_distance(facial_pos[v[0]], facial_pos[v[1]])
        
    return keypoint

def normalize_dict(data, max_value):
    for k in data.keys():
        data[k] = round(data[k]/max_value,5)
    return data

def get_feature_vector( keypoint ):
    ff = extract_face_feature(keypoint)
    ff = normalize_dict(ff, ff['face_height'])
    return ff

In [7]:
import json
def extract_face_feature_raw(facial_info):
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    dx = facial_pos['contour_left1']['x']
    dy = facial_pos['contour_left1']['y']
    ## calculate key point
    for (k,v) in facial_pos.items(): ## iteritems() for python2.7
        keypoint[k+'_x'] = v['x'] - dx
        keypoint[k+'_y'] = v['y'] - dy
        
    return keypoint

keypoint = extract_face_landmark('http://historie.kr/img/face/goodvc_01.jpg')
extract_face_feature_raw(keypoint)

{u'contour_chin_x': 7.608957999999994,
 u'contour_chin_y': 15.183194,
 u'contour_left1_x': 0.0,
 u'contour_left1_y': 0.0,
 u'contour_left2_x': -0.26354200000000105,
 u'contour_left2_y': 2.262806000000001,
 u'contour_left3_x': -0.023541999999999064,
 u'contour_left3_y': 4.442360999999998,
 u'contour_left4_x': 0.3097919999999945,
 u'contour_left4_y': 6.532361000000002,
 u'contour_left5_x': 0.8939579999999978,
 u'contour_left5_y': 8.645139,
 u'contour_left6_x': 1.8508329999999944,
 u'contour_left6_y': 10.322083,
 u'contour_left7_x': 2.9281249999999943,
 u'contour_left7_y': 12.045417,
 u'contour_left8_x': 4.165208,
 u'contour_left8_y': 13.544860999999997,
 u'contour_left9_x': 5.683957999999997,
 u'contour_left9_y': 14.783472000000003,
 u'contour_right1_x': 14.785832999999997,
 u'contour_right1_y': -1.4760279999999995,
 u'contour_right2_x': 14.994374999999998,
 u'contour_right2_y': 0.7420560000000016,
 u'contour_right3_x': 14.932707999999998,
 u'contour_right3_y': 3.4568890000000003,
 u'con

In [8]:
img_names = """
http://historie.kr/img/face/GyuseungKyeong_01.JPG
http://historie.kr/img/face/GyuseungKyeong_02.JPG
http://historie.kr/img/face/GyuseungKyeong_03.JPG
http://historie.kr/img/face/beomheepark_01.jpg
http://historie.kr/img/face/beomheepark_02.jpg
http://historie.kr/img/face/beomheepark_03.jpg
http://historie.kr/img/face/daesung_01.jpg
http://historie.kr/img/face/daesung_02.jpeg
http://historie.kr/img/face/daesung_03.jpg
http://historie.kr/img/face/eunjeeshin_01.jpg
http://historie.kr/img/face/eunjeeshin_02.jpg
http://historie.kr/img/face/eunjeeshin_03.jpg
http://historie.kr/img/face/eunjung_01.jpg
http://historie.kr/img/face/eunjung_02.jpg
http://historie.kr/img/face/eunjung_03.jpg
http://historie.kr/img/face/eunseop_01.jpg
http://historie.kr/img/face/eunseop_02.jpg.jpg
http://historie.kr/img/face/eunseop_03.jpg.jpg
http://historie.kr/img/face/gayeong_01.jpg
http://historie.kr/img/face/gayeong_02.jpg.jpg
http://historie.kr/img/face/gayeong_03.jpg.jpg
http://historie.kr/img/face/geeyeol_01.jpg
http://historie.kr/img/face/geeyeol_02.jpg
http://historie.kr/img/face/geeyeol_03.JPG
http://historie.kr/img/face/goodvc_01.jpg
http://historie.kr/img/face/goodvc_02.jpg
http://historie.kr/img/face/goodvc_03.jpg
http://historie.kr/img/face/hansung_01.jpg
http://historie.kr/img/face/hansung_02.jpg
http://historie.kr/img/face/hansung_03.jpg
http://historie.kr/img/face/jaekeun_01.jpg
http://historie.kr/img/face/jaekeun_02.jpg
http://historie.kr/img/face/jaekeun_03.jpg
http://historie.kr/img/face/jaeyoonyim_01.jpg
http://historie.kr/img/face/jaeyoonyim_02.jpg
http://historie.kr/img/face/jaeyoonyim_03.JPEG
http://historie.kr/img/face/jonghwan_01.jpg
http://historie.kr/img/face/jonghwan_03.jpg
http://historie.kr/img/face/jppark_02.jpeg
http://historie.kr/img/face/jypark_01.png
http://historie.kr/img/face/jypark_03.jpg
http://historie.kr/img/face/kimtaeyoun_01.jpg
http://historie.kr/img/face/kimtaeyoun_02.jpg
http://historie.kr/img/face/kimtaeyoun_03.png
http://historie.kr/img/face/landmark_raw.jpg
http://historie.kr/img/face/leeyongseok_01.jpg
http://historie.kr/img/face/leeyongseok_02.jpg
http://historie.kr/img/face/leeyongseok_03.jpg
http://historie.kr/img/face/namyoung_01.jpg
http://historie.kr/img/face/namyoung_02.jpg
http://historie.kr/img/face/namyoung_03.jpg
http://historie.kr/img/face/sechangoh_01.jpeg
http://historie.kr/img/face/sechangoh_02.jpeg
http://historie.kr/img/face/sechangoh_03.jpeg
http://historie.kr/img/face/sunghwan_01.jpg
http://historie.kr/img/face/sunghwan_02.jpg
http://historie.kr/img/face/sunghwan_03.jpg
http://historie.kr/img/face/yangchibok_01.jpg
http://historie.kr/img/face/yangchibok_02.jpg
http://historie.kr/img/face/yangchibok_03.jpg
http://historie.kr/img/face/yanghwi_01.png
http://historie.kr/img/face/yanghwi_02.jpg
http://historie.kr/img/face/yanghwi_03.jpg
"""

In [9]:
## (참고용) ls -al | awk '{ print "http://historie.kr/img/face/"$9}' 

image_data = {  'wonbin:1' : 'http://image.fmkorea.com/files/attach/new/20141213/486616/85599953/86802006/da08e25010e6001bf565cd5cca3ca359.jpg'
                , 'wonbin:2' : 'http://cfile25.uf.tistory.com/image/1907CA414F11DCF31C96F2'
                , 'wonbin:3' : 'http://cfile7.uf.tistory.com/image/145E12504DAF7CEE198138'
                , 'kimsoohyun:1' : 'http://cfile10.uf.tistory.com/image/167718454EB74A8C07D3DA'
                , 'kimsoohyun:2' : 'http://image.ytn.co.kr/general/jpg/2013/1218/201312181042081641_h.jpg'
                , 'kimsoohyun:3' : 'http://news.hankyung.com/nas_photo/201205/2012050296441_2012050290011.jpg'
              }


for link in img_names.split():
    name = (link.split('/')[-1].split('.')[0]).replace('_',':')
    image_data[name]= link

In [10]:
for n, u in image_data.items():
    print (n,u)

jaeyoonyim:03 http://historie.kr/img/face/jaeyoonyim_03.JPEG
sechangoh:03 http://historie.kr/img/face/sechangoh_03.jpeg
jppark:02 http://historie.kr/img/face/jppark_02.jpeg
kimtaeyoun:03 http://historie.kr/img/face/kimtaeyoun_03.png
eunjung:02 http://historie.kr/img/face/eunjung_02.jpg
eunjung:03 http://historie.kr/img/face/eunjung_03.jpg
eunjung:01 http://historie.kr/img/face/eunjung_01.jpg
yanghwi:01 http://historie.kr/img/face/yanghwi_01.png
GyuseungKyeong:01 http://historie.kr/img/face/GyuseungKyeong_01.JPG
kimtaeyoun:01 http://historie.kr/img/face/kimtaeyoun_01.jpg
GyuseungKyeong:03 http://historie.kr/img/face/GyuseungKyeong_03.JPG
GyuseungKyeong:02 http://historie.kr/img/face/GyuseungKyeong_02.JPG
yangchibok:01 http://historie.kr/img/face/yangchibok_01.jpg
yangchibok:03 http://historie.kr/img/face/yangchibok_03.jpg
leeyongseok:03 http://historie.kr/img/face/leeyongseok_03.jpg
leeyongseok:02 http://historie.kr/img/face/leeyongseok_02.jpg
leeyongseok:01 http://historie.kr/img/face/

In [11]:
landmark_dict = {}
for name, img_url in image_data.items(): 
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict[name] = get_feature_vector(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

extract fail : jppark:02
extract fail : yangchibok:01
extract fail : leeyongseok:03
extract fail : jaekeun:03
extract fail : jaekeun:02
extract fail : geeyeol:01
extract fail : eunseop:01


In [12]:
import pandas as pd
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')

In [13]:
landmark_ds.head()

,eye_to_eye,eye_width,lip_to_chin,face_width,eyebrow_width,face_height,nose_width,mouth_width,lip_to_nose
GyuseungKyeong:01,0.60659,0.84433,0.17473,0.31497,0.05844,1,0.37294,0.43512,0.17407
GyuseungKyeong:02,0.67471,0.95712,0.19845,0.36870,0.06168,1,0.44941,0.53248,0.16256
GyuseungKyeong:03,0.54750,0.77295,0.20055,0.28634,0.05422,1,0.36095,0.45304,0.18315
beomheepark:01,0.61961,0.87665,0.23048,0.33601,0.05524,1,0.36426,0.50411,0.20119
beomheepark:02,0.73309,1.05742,0.29486,0.47260,0.05732,1,0.49046,0.68046,0.23118


# raw facial landmark

In [14]:
## write to csv file
landmark_ds.to_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark.csv')

In [15]:
## load from csv file
landmark_ds = pd.DataFrame.from_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark.csv')

In [16]:
landmark_dict_raw = {}
for name, img_url in image_data.items():
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict_raw[name] = extract_face_feature_raw(keypoint)
    except:
        print ('extract fail : {0}'.format(name), img_url)

extract fail : jppark:02 http://historie.kr/img/face/jppark_02.jpeg
extract fail : yangchibok:01 http://historie.kr/img/face/yangchibok_01.jpg
extract fail : leeyongseok:03 http://historie.kr/img/face/leeyongseok_03.jpg
extract fail : jaekeun:03 http://historie.kr/img/face/jaekeun_03.jpg
extract fail : jaekeun:02 http://historie.kr/img/face/jaekeun_02.jpg
extract fail : geeyeol:01 http://historie.kr/img/face/geeyeol_01.jpg
extract fail : eunseop:01 http://historie.kr/img/face/eunseop_01.jpg


In [17]:
import pandas as pd
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')
landmark_ds.to_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark.csv')

In [18]:
import pandas as pd
## dict to dataframe
landmark_raw_ds = pd.DataFrame.from_dict( landmark_dict_raw, orient='index')
landmark_raw_ds.to_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark_raw.csv')

In [19]:
landmark_raw_ds.head(5).T

,GyuseungKyeong:01,GyuseungKyeong:02,GyuseungKyeong:03,beomheepark:01,beomheepark:02
contour_right5_y,16.035834,13.741000,9.305000,15.018518,15.569565
contour_right5_x,32.626888,32.658889,19.332666,37.757967,51.228913
mouth_upper_lip_left_contour3_x,12.810888,14.642445,7.204444,16.774634,23.476949
mouth_upper_lip_left_contour3_y,15.509500,14.335500,8.062166,16.167407,14.064272
left_eye_upper_left_quarter_y,-0.541500,0.218167,-0.386500,-0.518519,-0.747448
left_eye_upper_left_quarter_x,7.384000,9.124000,3.865777,8.428967,12.177251
contour_right7_y,21.664834,17.762500,12.676833,20.919259,22.362192
right_eyebrow_lower_right_quarter_y,-2.844833,-1.564500,-1.747334,-3.110618,-4.495275
left_eye_lower_right_quarter_x,10.930000,12.571333,5.942889,12.292967,17.739789
left_eye_lower_right_quarter_y,1.480000,1.740334,0.456833,1.076790,0.390359


In [20]:
for name, url in image_data.items():
    print(name, url )

jaeyoonyim:03 http://historie.kr/img/face/jaeyoonyim_03.JPEG
sechangoh:03 http://historie.kr/img/face/sechangoh_03.jpeg
jppark:02 http://historie.kr/img/face/jppark_02.jpeg
kimtaeyoun:03 http://historie.kr/img/face/kimtaeyoun_03.png
eunjung:02 http://historie.kr/img/face/eunjung_02.jpg
eunjung:03 http://historie.kr/img/face/eunjung_03.jpg
eunjung:01 http://historie.kr/img/face/eunjung_01.jpg
yanghwi:01 http://historie.kr/img/face/yanghwi_01.png
GyuseungKyeong:01 http://historie.kr/img/face/GyuseungKyeong_01.JPG
kimtaeyoun:01 http://historie.kr/img/face/kimtaeyoun_01.jpg
GyuseungKyeong:03 http://historie.kr/img/face/GyuseungKyeong_03.JPG
GyuseungKyeong:02 http://historie.kr/img/face/GyuseungKyeong_02.JPG
yangchibok:01 http://historie.kr/img/face/yangchibok_01.jpg
yangchibok:03 http://historie.kr/img/face/yangchibok_03.jpg
leeyongseok:03 http://historie.kr/img/face/leeyongseok_03.jpg
leeyongseok:02 http://historie.kr/img/face/leeyongseok_02.jpg
leeyongseok:01 http://historie.kr/img/face/